In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
print(nltk.data.path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
nltk.data.path.append('/root/nltk_data')  # Explicitly set the NLTK data path
nltk.download('punkt', download_dir='/root/nltk_data')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Veracity Prediction

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import DataLoader, Dataset, Subset
import pandas as pd

class FactVerificationDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        encoded_input = self.tokenizer.encode_plus(
            item['Claim'],
            item['Most Relevant Sentence'],
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )
        label = item['label']
        return {
            'input_ids': encoded_input['input_ids'].squeeze(),
            'attention_mask': encoded_input['attention_mask'].squeeze(),
            'label': torch.tensor(label)
        }

In [ ]:
# Load data
excel_file = 'RECTIFY Dataset/sentence.xlsx'
df = pd.read_excel(excel_file)
fields = ['Claim', 'Most Relevant Sentence', 'label']
data_list = df[fields].to_dict(orient='records')

In [ ]:
dataset = FactVerificationDataset(data_list)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import KFold

# Set up k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Store results
all_accuracies = []
all_reports = []

In [ ]:
import time  # Import time module
total_start_time = time.time()

In [ ]:
for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f"\nFold {fold + 1}/5")

    # Track training time per fold
    fold_start_time = time.time()

    train_subset = Subset(dataset, train_idx)
    test_subset = Subset(dataset, test_idx)
    train_dataloader = DataLoader(train_subset, batch_size=16, shuffle=True)
    test_dataloader = DataLoader(test_subset, batch_size=16, shuffle=False)

    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3).to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)


Fold 1/5


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Fold 2/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Fold 4/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Fold 5/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
  # Train model
    num_epochs = 5
    for epoch in range(num_epochs):
        epoch_start_time = time.time()  # Track epoch start time
        model.train()
        total_loss = 0

        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        epoch_time = time.time() - epoch_start_time  # Time taken for epoch
        print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {total_loss / len(train_dataloader):.4f} - Time: {epoch_time:.2f}s")

Epoch 1/5 - Loss: 0.3864 - Time: 348.56s
Epoch 2/5 - Loss: 0.1346 - Time: 364.19s
Epoch 3/5 - Loss: 0.0784 - Time: 364.59s
Epoch 4/5 - Loss: 0.0468 - Time: 363.97s
Epoch 5/5 - Loss: 0.0444 - Time: 364.15s
Epoch 1/5 - Loss: 0.0171 - Time: 363.74s
Epoch 2/5 - Loss: 0.0295 - Time: 363.84s
Epoch 3/5 - Loss: 0.0276 - Time: 363.53s
Epoch 4/5 - Loss: 0.0080 - Time: 363.78s
Epoch 5/5 - Loss: 0.0063 - Time: 363.88s
Epoch 1/5 - Loss: 0.0342 - Time: 364.01s
Epoch 2/5 - Loss: 0.0261 - Time: 363.73s
Epoch 3/5 - Loss: 0.0151 - Time: 364.05s
Epoch 4/5 - Loss: 0.0032 - Time: 364.12s
Epoch 5/5 - Loss: 0.0258 - Time: 364.13s
Epoch 1/5 - Loss: 0.0174 - Time: 363.67s
Epoch 2/5 - Loss: 0.0122 - Time: 363.32s
Epoch 3/5 - Loss: 0.0069 - Time: 363.50s
Epoch 4/5 - Loss: 0.0065 - Time: 363.60s
Epoch 5/5 - Loss: 0.0274 - Time: 363.84s
Epoch 1/5 - Loss: 0.0172 - Time: 363.44s
Epoch 2/5 - Loss: 0.0148 - Time: 363.19s
Epoch 3/5 - Loss: 0.0129 - Time: 363.64s
Epoch 4/5 - Loss: 0.0070 - Time: 363.82s
Epoch 5/5 - Loss

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    # Evaluate model
    eval_start_time = time.time()
    model.eval()
    correct_predictions = []
    total_predictions = []

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predicted_labels = torch.argmax(outputs.logits, dim=1)

            correct_predictions.extend(predicted_labels.cpu().numpy().tolist())
            total_predictions.extend(labels.cpu().numpy().tolist())

    eval_time = time.time() - eval_start_time  # Time taken for evaluation
    accuracy = accuracy_score(correct_predictions, total_predictions)

    fold_time = time.time() - fold_start_time  # Total time per fold
    print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f} - Training Time: {fold_time:.2f}s - Eval Time: {eval_time:.2f}s")

Fold 1 - Accuracy: 0.9590 - Training Time: 9108.00s - Eval Time: 27.46s
Fold 2 - Accuracy: 0.9590 - Training Time: 9135.47s - Eval Time: 27.46s
Fold 3 - Accuracy: 0.9590 - Training Time: 9162.95s - Eval Time: 27.48s
Fold 4 - Accuracy: 0.9590 - Training Time: 9190.48s - Eval Time: 27.53s
Fold 5 - Accuracy: 0.9590 - Training Time: 9218.06s - Eval Time: 27.57s


In [ ]:
# Total cross-validation time
total_time = time.time() - total_start_time
print(f"\nTotal Cross-validation Time: {total_time:.2f}s")


Total Cross-validation Time: 9222.65s
